In [3]:
import json
import os
from glob import glob
import numpy as np
from datetime import datetime

In [5]:
json_list = glob('F:/작업안전_주간보고/sample/*.json', recursive = True)
class_list = {}
with open('F:/작업안전_주간보고/class/warehouse_class.txt') as f:
    for i, j in enumerate(f):
        class_list[j.split('\n')[0]] = i
coco_file = open('F:/작업안전_주간보고/json_data/coco_file.txt', 'a', encoding='utf-8')

In [7]:
data_set = {}
data_set['info'] = {}
data_set['info']['description'] = 'WAREHOUSE'
data_set['info']['url'] = None
data_set['info']['version'] = '1.0'
data_set['info']['year'] = 2021
data_set['info']['contributor'] = None
data_set['info']['data_created'] = datetime.today().strftime('%Y-%m-%d')
data_set['licenses'] = {'url':None, 'id':0,'name':None}
data_set['images'] = []
data_set['type'] = 'instance'
data_set['annotation'] = []

In [10]:
for json_file in json_list:
    try :
        num = 0
        with open(json_file, encoding='utf-8') as j_file:
            json_data = json.load(j_file)
            img = json_data['Raw data Info.']
            img_info = {'lincese' : 1, 'url':None, 'file_name': json_data['Source data Info.']['source_data_ID']+'.jpg',
                        'height':img['resolution'][1], 'width':img['resolution'][0], 'date_captured':img['date'],
                        'id':num}
            ann_info = {'id':num, 'image_id':img_info['id']}

        num += 1
        data_set['images'].append(img_info)
    except Exception as e:
        print(e)
        print(json_file)
        pass


In [88]:
with open('F:/test/L-210827_G02_H_WS-22_001_0099.json', encoding='utf-8') as j_file:
    json_data = json.load(j_file)
    annotation = json_data['Learning data info.']['annotation']
    for i in annotation:
        if i['type']=='polygon':
            ann = i['data']

In [89]:
ann

[[1411.2119210705, 559.97909455233],
 [964.2816498521, 551.39271897344],
 [418.19071073432, 636.11085223493],
 [1342.2988388987, 722.98721300185],
 [1411.2119210705, 559.97909455233]]

In [13]:
np_ann = np.array(ann)
np_ann

array([[1411.21192107,  559.97909455],
       [ 964.28164985,  551.39271897],
       [ 418.19071073,  636.11085223],
       [1342.2988389 ,  722.987213  ],
       [1411.21192107,  559.97909455]])

In [ ]:
x=np_ann[:,0]
y=np_ann[:,1]
x,y

In [ ]:
area =0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
area

In [ ]:
index = np.argwhere(ann)
rows = index[:, 0]
clos = index[:, 1]

In [ ]:
rows, clos

In [ ]:
def mask2box(mask):
    index = np.argwhere(mask == 1)
    rows = index[:, 0]
    clos = index[:, 1]

    left_top_r = np.min(rows)  # y
    left_top_c = np.min(clos)  # x

    right_bottom_r = np.max(rows)
    right_bottom_c = np.max(clos)

    return [
        left_top_c,
        left_top_r,
        right_bottom_c - left_top_c,
        right_bottom_r - left_top_r,
    ]


In [ ]:
from PIL import Image
from PIL import ImageDraw

img_shape = [1920,1080]
polygons = ann
mask = np.zeros(img_shape, dtype=np.uint8)
mask = Image.fromarray(mask)
xy = list(map(tuple, polygons))
ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
mask = np.array(mask, dtype=bool)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def pil_draw_rect(image, point1, point2):
    draw = ImageDraw.Draw(image)
    draw.rectangle((point1, point2), outline=(0, 0, 255), width=3)

    return image
image = Image.open('F:/test/L-210827_G02_H_WS-22_001_0099.jpg')
image = pil_draw_rect(image, (30, 30), (230, 130))
plt.imshow(np.array(image))
plt.show()

In [ ]:
image = Image.open('F:/test/L-210827_G02_H_WS-22_001_0099.jpg')

polygons = ann
mask = np.zeros(image.size, dtype=np.uint8)
mask = Image.fromarray(mask)
xy = list(map(tuple, polygons))
draw = ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
mask = np.array(mask, dtype=bool)

In [ ]:
mask

In [78]:
draw = ImageDraw.Draw(image)
draw.line((aa,bb),+image.size, fill=128)


TypeError: bad operand type for unary +: 'tuple'

In [2]:

np_ann

NameError: name 'np_ann' is not defined

In [60]:
import cv2

test = np_ann.astype(np.int32)
# test = test.reshape((-1,1,2))
image = cv2.imread('F:/test/L-210827_G02_H_WS-22_001_0099.jpg')
cv2.polylines(image, [test],True,(0,255,255),5)
cv2.rectangle(image, (x.max(),y.min()), (x.min(),y.max()), (255,255,0),2)

cv2.imshow('test',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [65]:
# cv2.rectangle(image, (x.max(),y.max()), (x.min(),y.min()), (255,0,0),2)
x=np_ann[:,0]
y=np_ann[:,1]
x,y

(array([1411.21192107,  964.28164985,  418.19071073, 1342.2988389 ,
        1411.21192107]),
 array([559.97909455, 551.39271897, 636.11085223, 722.987213  ,
        559.97909455]))

In [53]:
(x.max(),y.min())

(1411, 551)

In [54]:
x.min(),y.max()

(418, 722)

In [55]:
(x.max(),y.max()), (x.min(),y.min())

((1411, 722), (418, 551))

In [81]:
def getbbox(points):
    polygons = np.array(points)
    x,y = polygons[:,0], polygons[:,1]
    bbox = [x.max(),y.min(), x.min(),y.max()]
    return bbox

In [82]:
aa = getbbox(ann)
aa

[1411.2119210705, 551.39271897344, 418.19071073432, 722.98721300185]

In [76]:
ann

[[1411.2119210705, 559.97909455233],
 [964.2816498521, 551.39271897344],
 [418.19071073432, 636.11085223493],
 [1342.2988388987, 722.98721300185],
 [1411.2119210705, 559.97909455233]]

In [78]:
polygons = np.array(ann)
polygons


array([[1411.21192107,  559.97909455],
       [ 964.28164985,  551.39271897],
       [ 418.19071073,  636.11085223],
       [1342.2988389 ,  722.987213  ],
       [1411.21192107,  559.97909455]])

In [90]:
def getbbox(points):
    polygons = np.array(points)
    x,y = polygons[:,0], polygons[:,1]
    bbox = [x.max(),y.min(), x.min(),y.max()]
    return bbox

def annotation(points):
    annotation = {}
    contour = np.array(points)
    x = contour[:, 0]
    y = contour[:, 1]
    area = 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))
    annotation["segmentation"] = [list(np.asarray(points).flatten())]
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = num

    annotation["bbox"] = getbbox(points)

    return annotation

In [91]:
aaa = annotation(ann)
aaa

{'segmentation': [[1411.2119210705,
   559.97909455233,
   964.2816498521,
   551.39271897344,
   418.19071073432,
   636.11085223493,
   1342.2988388987,
   722.98721300185,
   1411.2119210705,
   559.97909455233]],
 'iscrowd': 0,
 'area': 99588.04259173479,
 'image_id': 1,
 'bbox': [1411.2119210705, 551.39271897344, 418.19071073432, 722.98721300185]}

In [92]:
type(ann)


list

In [94]:
import os, shutil
from glob import glob

for i in glob('F:/작업안전_주간보고/image_data/*.jpg'):
    if os.path.exists('F:/작업안전_주간보고/json_data/'+i.split('\\')[-1].replace('.jpg','.json')):
        shutil.move('F:/작업안전_주간보고/json_data/'+i.split('\\')[-1].replace('.jpg','.json'),
                    'F:/작업안전_주간보고/sample/'+i.split('\\')[-1].replace('.jpg','.json'))
        shutil.move(i, i.replace('image_data','sample'))

In [98]:

data_set = {}
data_set['categories'] = []
with open('F:/작업안전_주간보고/class/warehouse_poly.txt') as f:
    for i, j in enumerate(f):
        class_list = {}
        class_list['supercategory'] = None
        class_list['id'] = i
        class_list['name'] = j.split('\n')[0]

        data_set['categories'].append(class_list)

In [99]:
data_set

{'categories': [{'supercategory': None, 'id': 0, 'name': 'WO-08'},
  {'supercategory': None, 'id': 1, 'name': 'SO-01'},
  {'supercategory': None, 'id': 2, 'name': 'SO-05'},
  {'supercategory': None, 'id': 3, 'name': 'SO-13'},
  {'supercategory': None, 'id': 4, 'name': 'SO-14'},
  {'supercategory': None, 'id': 5, 'name': 'SO-15'}]}

In [100]:
with open('F:/작업안전_주간보고/class/warehouse_bbox.txt') as f:
    for i, j in enumerate(f):
        print(j)
        box_list[j.split('\n')[0]] = i

UA-01



NameError: name 'box_list' is not defined

In [106]:
def class_dict(file_list):
    classes = {}
    for i, j in enumerate(file_list):
        classes[j.split('\n')[0]] = i
    return classes

In [107]:
with open('F:/작업안전_주간보고/class/warehouse_bbox.txt',encoding='utf-8') as f:
    aa = class_dict(f)
aa

{'UA-01': 0,
 'UA-03': 1,
 'UA-04': 2,
 'UA-05': 3,
 'UA-06': 4,
 'UA-09': 5,
 'UA-10': 6,
 'UA-12': 7,
 'UA-13': 8,
 'UA-14': 9,
 'UA-15': 10,
 'UA-16': 11,
 'UA-17': 12,
 'UA-20': 13,
 'UC-02': 14,
 'UC-03': 15,
 'UC-06': 16,
 'UC-08': 17,
 'UC-10': 18,
 'UC-13': 19,
 'UC-14': 20,
 'UC-15': 21,
 'UC-16': 22,
 'UC-17': 23,
 'UC-18': 24,
 'UC-19': 25,
 'UC-20': 26,
 'UC-21': 27,
 'UC-22': 28,
 'WO-01': 29,
 'WO-02': 30,
 'WO-03': 31,
 'WO-04': 32,
 'WO-05': 33,
 'WO-06': 34,
 'WO-07': 35,
 'SO-02': 36,
 'SO-03': 37,
 'SO-04': 38,
 'SO-06': 39,
 'SO-07': 40,
 'SO-08': 41,
 'SO-09': 42,
 'SO-10': 43,
 'SO-11': 44,
 'SO-12': 45,
 'SO-16': 46,
 'SO-17': 47,
 'SO-19': 48,
 'SO-20': 49,
 'SO-21': 50,
 'SO-22': 51,
 'SO-23': 52}

In [116]:
from datetime import datetime
num = 0
data_set = {}
data_set['info'] = {}
data_set['info'] = {'description': 'WAREHOUSE', 'url': None, 'version': '1.0',
                    'year':datetime.today().year, 'contributor':None,
                    'data_created':datetime.today().strftime('%Y-%m-%d')}
data_set['licenses'] = {'url':None, 'id':0,'name':None}
data_set['images'] = []
data_set['type'] = 'instance'
data_set['annotation'] = []
data_set['categories'] = []

In [117]:
data_set

{'info': {'description': 'WAREHOUSE',
  'url': None,
  'version': '1.0',
  'year': 2021,
  'contributor': None,
  'data_created': '2021-09-25'},
 'licenses': {'url': None, 'id': 0, 'name': None},
 'images': [],
 'type': 'instance',
 'annotation': [],
 'categories': []}

In [118]:
with open('F:/작업안전_주간보고/class/warehouse_poly.txt') as f:
    poly_list = class_dict(f)
    for i, j in enumerate(f):
        class_list.update({'supercategory': None, 'id': i, 'name': j.split('\n')[0]})
        print(class_list)

In [123]:
def class_dict(file_list):
    classes = {}
    class_list = []
    for i, j in enumerate(file_list):
        classes[j.split('\n')[0]] = i
        class_list.append({'supercategory': None, 'id': i, 'name': j.split('\n')[0]})
    return classes, class_list

In [124]:
data_set ={}
with open('F:/작업안전_주간보고/class/warehouse_poly.txt') as f:
    poly_list, data_set['categories'] = class_dict(f)


In [125]:
poly_list

{'WO-08': 0, 'SO-01': 1, 'SO-05': 2, 'SO-13': 3, 'SO-14': 4, 'SO-15': 5}

In [126]:
data_set

{'categories': [{'supercategory': None, 'id': 0, 'name': 'WO-08'},
  {'supercategory': None, 'id': 1, 'name': 'SO-01'},
  {'supercategory': None, 'id': 2, 'name': 'SO-05'},
  {'supercategory': None, 'id': 3, 'name': 'SO-13'},
  {'supercategory': None, 'id': 4, 'name': 'SO-14'},
  {'supercategory': None, 'id': 5, 'name': 'SO-15'}]}